In [4]:
# ==========================================================
# High-Fidelity Artwork Analyzer v13.5 (Deep Fidelity)
# ==========================================================
# Local-only: OpenCV + NumPy + Pillow
# Outputs per image:
#   /<image>_analysis/
#     ├─ overlay_final_composite.png
#     ├─ curatorial_summary.txt
#     └─ metrics.json
# ==========================================================

import os, json, time, textwrap, math
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees

# -----------------------------
# Utilities
# -----------------------------
def load_rgb(path):
    img = Image.open(path).convert("RGB")
    return np.array(img)

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_uint8(img_f32): 
    return np.clip(img_f32*255.0, 0, 255).astype(np.uint8)

def safe_gamma(img_f32):
    m = float(img_f32.mean())
    if m < 0.20:
        img_f32 = np.power(img_f32, 0.6)   # brighten dark
    elif m > 0.85:
        img_f32 = np.power(img_f32, 1.2)   # compress bright
    return np.clip(img_f32, 0, 1)

def resize_min_pixels(img, target_pixels=1_800_000):
    H, W = img.shape[:2]
    cur = H*W
    if cur >= target_pixels:
        return img, 1.0
    scale = math.sqrt(target_pixels/float(cur))
    newW, newH = int(W*scale), int(H*scale)
    new = cv2.resize(img, (newW, newH), interpolation=cv2.INTER_CUBIC)
    return new, scale

# -----------------------------
# JSON-safe conversion
# -----------------------------
def make_json_safe(obj):
    if isinstance(obj, dict):
        return {k: make_json_safe(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [make_json_safe(x) for x in obj]
    elif isinstance(obj, (np.generic, np.number)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# -----------------------------
# Legend bars (hue & orientation)
# -----------------------------
def make_hue_bar(hist, width=420, height=50):
    # hist: 36-bin circular hue histogram (0..360)
    bar = np.zeros((height, width, 3), np.uint8)
    for x in range(width):
        h = int((x/width)*179)
        rgb = cv2.cvtColor(np.uint8([[[h, 200, 220]]]), cv2.COLOR_HSV2RGB)[0,0]
        bar[:, x] = rgb
    bins = len(hist)
    if bins <= 0: 
        return bar
    bin_w = max(1, width // bins)
    norm = hist.astype(np.float32)
    if norm.sum() > 0:
        norm /= (norm.max() + 1e-6)
    for i, v in enumerate(norm):
        x1 = i*bin_w
        x2 = min(width-1, (i+1)*bin_w-1)
        h_px = int(v*(height-10))
        cv2.rectangle(bar, (x1, height-1), (x2, height-1-h_px), (0,0,0), -1)
    return bar

def make_orientation_bar(angles, energies, width=420, height=50):
    bar = np.full((height, width, 3), 240, np.uint8)
    if energies is None or len(energies) == 0:
        return bar
    e = np.array(energies, dtype=np.float32)
    e = (e - e.min())/(e.max()-e.min()+1e-6)
    for a, v in zip(angles, e):
        x = int((a/180.0)*width)
        h_px = int(v*(height-10))
        cv2.rectangle(bar, (x-2, height-1), (x+2, height-1-h_px), (60,60,60), -1)
    cv2.putText(bar, "0°", (5, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "90°", (width//2-15, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "180°", (width-50, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    return bar
# -----------------------------
# CIEDE2000 (pairwise for cluster centers)
# -----------------------------
def ciede2000(Lab1, Lab2):
    L1,a1,b1 = Lab1; L2,a2,b2 = Lab2
    C1 = np.sqrt(a1*a1 + b1*b1); C2 = np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2)/2.0
    G = 0.5*(1 - np.sqrt((avg_C**7)/((avg_C**7)+25**7)))
    a1p, a2p = (1+G)*a1, (1+G)*a2
    C1p, C2p = np.sqrt(a1p*a1p + b1*b1), np.sqrt(a2p*a2p + b2*b2)
    h1p = (np.degrees(np.arctan2(b1, a1p)) + 360) % 360
    h2p = (np.degrees(np.arctan2(b2, a2p)) + 360) % 360
    dLp = L2-L1; dCp = C2p-C1p
    dhp = h2p-h1p
    if C1p*C2p == 0: dhp = 0
    else:
        if dhp > 180: dhp -= 360
        elif dhp < -180: dhp += 360
    dHp = 2*np.sqrt(C1p*C2p)*np.sin(np.radians(dhp)/2.0)
    avg_Lp = (L1+L2)/2.0
    if C1p*C2p == 0:
        avg_hp = h1p + h2p
    else:
        if abs(h1p-h2p) > 180:
            avg_hp = (h1p + h2p + 360)/2.0 if (h1p+h2p) < 360 else (h1p + h2p - 360)/2.0
        else:
            avg_hp = (h1p + h2p)/2.0
    T = (1 - 0.17*np.cos(np.radians(avg_hp-30)) +
         0.24*np.cos(np.radians(2*avg_hp)) +
         0.32*np.cos(np.radians(3*avg_hp+6)) -
         0.20*np.cos(np.radians(4*avg_hp-63)))
    d_ro = 30*np.exp(-((avg_hp-275)/25)**2)
    Rc = 2*np.sqrt(( ( ( ( (avg_C := (C1p+C2p)/2.0) )**7 ) / ((avg_C**7)+25**7) ) ))
    Sl = 1 + 0.015*(avg_Lp-50)**2 / np.sqrt(20 + (avg_Lp-50)**2)
    Sc = 1 + 0.045*avg_C
    Sh = 1 + 0.015*avg_C*T
    Rt = -np.sin(np.radians(2*d_ro))*Rc
    return float(np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh)))

def _kmeans_lab(lab_img, k, attempts=10):
    H, W = lab_img.shape[:2]
    Z = lab_img.reshape(-1,3).astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 80, 0.2)
    _r, labels, centers = cv2.kmeans(Z, k, None, crit, attempts, cv2.KMEANS_PP_CENTERS)
    labels2 = labels.reshape(H, W)
    return labels2, centers.astype(np.float32)

def _palette_quality(centers):
    # combine entropy (area distribution unknown here) + mean ΔE proxy among centers
    # fallback uses variance of centers as spread metric + mean ΔE among pairwise centers
    if centers.shape[0] < 2:
        return 0.0, 0.0
    # ΔE across centers
    dEs = [ciede2000(centers[i], centers[j]) for i in range(len(centers)) for j in range(i+1, len(centers))]
    mean_dE = float(np.mean(dEs)) if dEs else 0.0
    spread = float(np.std(centers, axis=0).mean())
    score = 0.7*mean_dE + 0.3*spread
    return score, mean_dE

def analyze_color_deep_ensemble(img, ks=(6,8,10)):
    H, W = img.shape[:2]
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    best = None
    best_score = -1
    results = {}

    # hue histogram for legend
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hue = (hsv[:,:,0].astype(np.float32) * 2.0) % 360.0
    hue_hist, _ = np.histogram(hue, bins=36, range=(0,360))
    hue_hist = hue_hist.astype(np.float32)

    for k in ks:
        labels2, centers = _kmeans_lab(lab, k)
        counts = np.bincount(labels2.reshape(-1), minlength=k).astype(np.float32)
        masks = [(labels2 == i).astype(np.uint8)*255 for i in range(k)]
        # entropy using area fractions
        frac = counts/(H*W + 1e-6)
        p = frac[frac > 0]
        entropy = -float(np.sum(p*np.log2(p)))
        score, mean_dE = _palette_quality(centers)
        quality = 0.6*score + 0.4*entropy  # composite quality

        results[k] = dict(centers=centers, masks=masks, counts=counts, entropy=entropy, mean_dE=mean_dE)

        if quality > best_score:
            best_score = quality
            best = (k, labels2, centers, masks, counts, entropy, mean_dE)

    k_sel, labels2, centers, masks, counts, entropy, mean_dE = best
    rgb_centers = cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8),
                               cv2.COLOR_Lab2RGB).reshape(-1,3)
    palette = [tuple(map(int, c)) for c in rgb_centers]

    a, b = centers[:,1]-128, centers[:,2]-128
    hues_c = (np.degrees(np.arctan2(b, a)) + 360) % 360
    warm_ratio = float(((hues_c < 90) | (hues_c > 330)).sum()/len(hues_c))
    cool_ratio = float(((hues_c > 180) & (hues_c < 300)).sum()/len(hues_c))
    harmony = "mixed"
    if np.ptp(hues_c) < 35: harmony = "analogous"
    elif any(abs(hues_c[i]-hues_c[j])%360 > 150 for i in range(len(hues_c)) for j in range(i+1, len(hues_c))):
        harmony = "complementary"

    # ΔE proxy field from Lab gradients
    labf = lab.astype(np.float32)
    gxL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 1, 0); gyL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 0, 1)
    gxa = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 1, 0); gya = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 0, 1)
    gxb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 1, 0); gyb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 0, 1)
    grad_mag = np.sqrt((gxL**2 + gyL**2) + (gxa**2 + gya**2) + (gxb**2 + gyb**2))
    contrast_map = (grad_mag - grad_mag.min())/(grad_mag.max()-grad_mag.min()+1e-6)

    return dict(
        palette_rgb=palette,
        masks=masks,
        spatial_cohesion=float((counts/(H*W+1e-6)).max()),
        palette_entropy=float(entropy),
        mean_deltaE=float(mean_dE),
        max_deltaE=float(np.max([ciede2000(centers[i], centers[j]) 
                                 for i in range(len(centers)) for j in range(i+1,len(centers))])) if len(centers)>1 else 0.0,
        warm_ratio=warm_ratio,
        cool_ratio=cool_ratio,
        harmony=harmony,
        contrast_map=contrast_map,
        hue_hist=hue_hist,
        k_selected=int(k_sel),
        ks=list(ks)
    )
# -----------------------------
# Texture: Gabor pyramid (scales 3..13, orientations 12)
# -----------------------------
def gabor_energy_pyramid(gray, scales=(3,5,7,9,11,13), orientations=12):
    H, W = gray.shape
    angles = np.linspace(0, 180, orientations, endpoint=False)
    combined = np.zeros((H,W), np.float32)
    angle_energy = np.zeros(orientations, np.float32)

    for oi, theta in enumerate(angles):
        theta_rad = np.deg2rad(theta)
        e_sum = np.zeros((H,W), np.float32)
        for k in scales:
            kernel = cv2.getGaborKernel((21,21), sigma=k, theta=theta_rad,
                                        lambd=10, gamma=0.5, psi=0)
            resp = cv2.filter2D(gray, cv2.CV_32F, kernel)
            e_sum += resp*resp
        combined = np.maximum(combined, e_sum)
        angle_energy[oi] = e_sum.mean()

    anisotropy = float((angle_energy.max() - angle_energy.min())/(angle_energy.mean()+1e-6))
    dom_angle = float(angles[int(np.argmax(angle_energy))])
    gmap = (combined - combined.min())/(combined.max()-combined.min()+1e-6)

    # entropy & roughness proxies
    hist, _ = np.histogram(gray, bins=256, range=(0,256))
    hist = hist.astype(np.float32); hist /= (hist.sum()+1e-6)
    entropy = -float(np.sum(hist*np.log2(hist + 1e-12)))
    roughness = float(gray.std())

    return dict(
        gabor_angles=angles,
        gabor_energy=angle_energy,
        gabor_map=gmap,
        gabor_anisotropy=anisotropy,
        gabor_dom_angle=dom_angle,
        lbp_entropy=entropy,
        lbp_roughness=roughness
    )

# -----------------------------
# Saliency: two-scale spectral residual + mask fusion
# -----------------------------
def spectral_saliency(grayf, blur=(9,9), sigma=2):
    fft = np.fft.fft2(grayf)
    loga = np.log(np.abs(fft)+1e-8)
    spec = np.exp((loga - cv2.blur(loga,(3,3))) + 1j*np.angle(fft))
    sal = np.abs(np.fft.ifft2(spec))**2
    sal = cv2.GaussianBlur(sal, blur, sigma)
    sal = (sal - sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency_fused(s1, s2):
    H, W = s1.shape
    m1 = cv2.threshold(s1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    m2 = cv2.threshold(s2, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    m = cv2.bitwise_and(m1, m2)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))
    cnts,_ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h = cv2.boundingRect(max(cnts, key=cv2.contourArea))
    else:
        x,y,w,h = W//4, H//4, W//2, H//2
        cv2.rectangle(m, (x,y), (x+w, y+h), 255, -1)
    return dict(mask=m, bbox=(x,y,w,h))

def analyze_saliency_deep(img):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float32)
    s_coarse = spectral_saliency(cv2.GaussianBlur(g,(0,0), 3.0), blur=(11,11), sigma=3)
    s_fine   = spectral_saliency(cv2.GaussianBlur(g,(0,0), 1.0), blur=(7,7),  sigma=1.5)
    fused = np.maximum(s_coarse, s_fine)
    sub = subject_from_saliency_fused(s_coarse, s_fine)
    Y, X = np.indices(fused.shape); w = fused.astype(np.float32)+1
    cx, cy = int((X*w).sum()/w.sum()), int((Y*w).sum()/w.sum())
    return dict(saliency_map=fused, focal=(cx,cy), subject_mask=sub["mask"], subject_bbox=sub["bbox"])

# -----------------------------
# Composition: multi-sweep Hough + vanishing point clustering
# -----------------------------
def line_intersection(p1, p2):
    (x1,y1,x2,y2), (x3,y3,x4,y4) = p1, p2
    den = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
    if abs(den) < 1e-6: return None
    px = ((x1*y2 - y1*x2)*(x3-x4) - (x1-x2)*(x3*y4 - y3*x4))/den
    py = ((x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4 - y3*x4))/den
    return (px, py)

def cluster_points(points, grid=40):
    # simple grid binning to find dense intersection area
    if not points: 
        return None
    pts = np.array(points, dtype=np.float32)
    xs, ys = pts[:,0], pts[:,1]
    xbin = np.round(xs/grid).astype(int)
    ybin = np.round(ys/grid).astype(int)
    keys = xbin*100000 + ybin
    uniq, counts = np.unique(keys, return_counts=True)
    best_key = uniq[np.argmax(counts)]
    bx = (best_key // 100000) * grid
    by = (best_key % 100000) * grid
    # average of points in the best bin
    mask = (keys == best_key)
    bx = float(xs[mask].mean())
    by = float(ys[mask].mean())
    return (bx, by)

def analyze_composition_deep(img, focal_xy):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    H, W = g.shape
    edges = cv2.Canny(g, 80, 200)

    lines = []
    # multi-sweep Hough
    params = [
        (1, np.pi/180, 100, min(H,W)//12, 15),
        (1, np.pi/180, 80,  min(H,W)//14, 20),
        (1, np.pi/180, 120, min(H,W)//10, 15)
    ]
    for rho, theta, thr, minL, gap in params:
        linesP = cv2.HoughLinesP(edges, rho, theta, thr, minLineLength=minL, maxLineGap=gap)
        if linesP is not None:
            for l in linesP:
                ln = tuple(map(int, l[0]))
                lines.append(ln)

    # deduplicate short or near-identical lines
    dedup = []
    for (x1,y1,x2,y2) in lines:
        if math.hypot(x2-x1, y2-y1) < min(H,W)//14: 
            continue
        dedup.append((x1,y1,x2,y2))
    lines = dedup[:300]

    # vanishing point via line intersections inside image bounds
    inters = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            ip = line_intersection(lines[i], lines[j])
            if ip is None: 
                continue
            if -W*0.5 <= ip[0] <= W*1.5 and -H*0.5 <= ip[1] <= H*1.5:
                inters.append(ip)
    vp = cluster_points(inters, grid=max(20, min(H,W)//30))
    if vp is None:
        vp = (W/2, H/2)

    # rule-of-thirds & golden deltas
    fx, fy = focal_xy
    thirds_pts = [(W/3,H/3),(2*W/3,H/3),(W/3,2*H/3),(2*W/3,2*H/3)]
    gfac = 0.618
    golden_pts = [(gfac*W,gfac*H),((1-gfac)*W,gfac*H),
                  (gfac*W,(1-gfac)*H),((1-gfac)*W,(1-gfac)*H)]
    diag = np.hypot(W,H)
    thirds_delta = min(np.hypot(fx-x,fy-y) for (x,y) in thirds_pts)/(diag+1e-6)
    golden_delta = min(np.hypot(fx-x,fy-y) for (x,y) in golden_pts)/(diag+1e-6)

    # symmetry & edge balance
    g_f = g.astype(np.float32)/255.0
    left = g_f[:, :W//2]
    right = np.fliplr(g_f[:, W - left.shape[1]:])
    mse = float(np.mean((left - right)**2))
    symmetry = float(1.0 - min(1.0, mse*4.0))
    lw, rw = float(np.sum(edges[:,:W//2])), float(np.sum(edges[:,W//2:]))
    balance = (rw - lw)/(lw + rw + 1e-6)

    return dict(
        lines=lines,
        vanishing_point=vp,
        thirds_delta=thirds_delta,
        golden_delta=golden_delta,
        symmetry=symmetry,
        edge_balance=balance
    )

# -----------------------------
# Illumination (direction + skew)
# -----------------------------
def illumination(img):
    L = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu, sig = L.mean(), L.std()+1e-6
    skew = float((((L-mu)/sig)**3).mean())
    gx, gy = cv2.Sobel(L, cv2.CV_32F, 1,0), cv2.Sobel(L, cv2.CV_32F, 0,1)
    hi = (L > np.percentile(L, 90)).astype(np.uint8)
    if hi.sum() > 0:
        gx, gy = (gx*hi).sum(), (gy*hi).sum()
        ang = float((degrees(atan2(-gy, -gx)) + 360) % 360)
    else:
        ang = None
    return dict(lightness_skew=skew, light_direction_deg=ang)
# -----------------------------
# Curatorial summary (encouraging)
# -----------------------------
def generate_curatorial_summary(M):
    C, T, P, I = M["color"], M["texture"], M["composition"], M["illumination"]
    tone = {"analogous":"a gentle, harmonious range",
            "complementary":"a lively conversation of opposites"}.get(C["harmony"], "a balanced and nuanced palette")
    if C["mean_deltaE"] > 25: contrast = "confident contrasts that energize the field"
    elif C["mean_deltaE"] > 10: contrast = "measured contrasts that keep the eye engaged"
    else: contrast = "soft transitions that encourage a calm reading"
    texture_phrase = (
        f"Texture carries a directional current (peak near {T['gabor_dom_angle']:.0f}°), guiding the eye"
        if T["gabor_anisotropy"] > 0.5 else
        "Texture feels even and poised, letting color and shape lead"
    )
    comp_phrase = (
        "The focal area resonates with classical proportion."
        if P["golden_delta"] < 0.12 else
        "The focal area settles near the rule-of-thirds, feeling intuitive and open."
        if P["thirds_delta"] < 0.12 else
        "The focal area resists fixed proportion, inviting free exploration."
    )
    light_phrase = (
        f"Light arrives around {int(I['light_direction_deg'])}°, gently shaping perception."
        if I["light_direction_deg"] is not None else
        "Light diffuses evenly, letting the whole surface breathe."
    )
    mood = "bright and open" if I["lightness_skew"] > 0.5 else \
           "quietly contemplative" if I["lightness_skew"] < -0.5 else \
           "balanced and steady"

    micro = textwrap.fill(
        " • Palette: " + tone + ". " +
        " • Contrast: " + contrast + ". " +
        " • Texture: " + ("directional" if T["gabor_anisotropy"]>0.5 else "even") + ". " +
        " • Composition: classical cues with flexible balance. " +
        " • Light: " + ("directional" if I["light_direction_deg"] else "diffuse") + ".",
        width=90
    )
    narrative = textwrap.fill(
        "This work invites an attentive gaze. The palette offers " + tone + ", with " + contrast +
        ". " + texture_phrase + ". " + comp_phrase + " " + light_phrase +
        " The mood feels " + mood + ", encouraging you to linger as color, gesture, and structure support one another.",
        width=90
    )
    return micro, narrative

# -----------------------------
# Composite with strong overlays + minimal legend
# -----------------------------
def build_composite(img, M):
    base = img.astype(np.float32)/255.0
    H, W = base.shape[:2]

    # 1) Color clusters (tinted wash)
    color_layer = np.zeros_like(base)
    for m, c in zip(M["color"]["masks"], M["color"]["palette_rgb"]):
        if m.sum() == 0: continue
        mask = cv2.GaussianBlur((m > 0).astype(np.float32), (0,0), 6)
        color_layer += np.dstack([mask]*3) * (np.array(c)/255.0)
    color_layer *= 0.14

    # 2) Texture (Gabor energy) → amber
    gmap = M["texture"]["gabor_map"]
    gmap = cv2.GaussianBlur(gmap, (0,0), 1.0)
    gmap = (gmap - gmap.min())/(gmap.max()-gmap.min()+1e-6)
    tex_layer = np.dstack([gmap, gmap*0.45, 0*gmap]) * 0.38

    # 3) ΔE proxy → magenta
    contrast = M["color"]["contrast_map"]
    contrast = cv2.GaussianBlur(contrast, (0,0), 1.0)
    contrast = (contrast - contrast.min())/(contrast.max()-contrast.min()+1e-6)
    de_layer = np.dstack([contrast*0.60, 0*contrast, contrast*0.60]) * 0.36

    # 4) Saliency → soft red
    sal = M["saliency"]["saliency_map"].astype(np.float32)
    if sal.shape[:2] != (H, W):
        sal = cv2.resize(sal, (W, H), interpolation=cv2.INTER_CUBIC)
    sal = cv2.normalize(sal, None, 0, 1, cv2.NORM_MINMAX)
    sal_layer = np.dstack([sal*0.70, sal*0.30, sal*0.30]) * 0.30

    # 5) Lines (cyan/green/yellow by angle)
    line_layer = np.zeros_like(base)
    for (x1,y1,x2,y2) in M["composition"]["lines"][:240]:
        ang = (np.rad2deg(np.arctan2(y2-y1, x2-x1)) + 180) % 180
        col = (0,1,0) if 60<ang<120 else (0,0,1) if ang<30 or ang>150 else (1,1,0)
        cv2.line(line_layer,(x1,y1),(x2,y2),(col[2],col[1],col[0]),1,cv2.LINE_AA)
    line_layer = cv2.GaussianBlur(line_layer, (0,0), 2) * 0.55

    comp = base*0.82 + color_layer + tex_layer + de_layer + sal_layer + line_layer
    comp = np.clip(comp, 0, 1)

    # Subject contour + glow and focal rings
    comp_bgr = (comp[:,:,::-1]*255).astype(np.uint8).copy()
    mask = M["saliency"]["subject_mask"]
    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        glow = np.zeros_like(comp_bgr)
        cv2.drawContours(glow, cnts, -1, (255, 0, 255), thickness=cv2.FILLED)
        glow = cv2.GaussianBlur(glow, (0,0), 15)
        comp_bgr = cv2.addWeighted(glow, 0.14, comp_bgr, 0.86, 0)
        cv2.drawContours(comp_bgr, cnts, -1, (255, 0, 255), 2)
    fx, fy = M["saliency"]["focal"]
    for r in range(6, 30, 6):
        cv2.circle(comp_bgr, (int(fx), int(fy)), r, (0,0,255), 1)

    # Thirds & Golden lines
    thirds_x, thirds_y = [W//3, 2*W//3], [H//3, 2*H//3]
    for x in thirds_x: cv2.line(comp_bgr, (x,0), (x,H), (240,240,240), 1)
    for y in thirds_y: cv2.line(comp_bgr, (0,y), (W,y), (240,240,240), 1)
    gfac = 0.618
    gx = [int(gfac*W), int((1-gfac)*W)]
    gy = [int(gfac*H), int((1-gfac)*H)]
    for x in gx: cv2.line(comp_bgr, (x,0), (x,H), (0,255,255), 1)
    for y in gy: cv2.line(comp_bgr, (0,y), (W,y), (0,255,255), 1)

    # Vanishing point + Light arrow
    vp = M["composition"].get("vanishing_point")
    if vp: cv2.circle(comp_bgr, tuple(map(int, vp)), 6, (0,255,255), -1)
    ang = M["illumination"].get("light_direction_deg")
    if ang is not None:
        cx, cy = W-60, 60; r = min(H,W)//3
        ex, ey = int(cx - r*np.cos(np.radians(ang))), int(cy + r*np.sin(np.radians(ang)))
        cv2.arrowedLine(comp_bgr, (cx,cy), (ex,ey), (0,150,255), 3, tipLength=0.1)

    comp_rgb = comp_bgr[:,:,::-1].astype(np.float32)/255.0

    # Legend (minimal text)
    legend_h = 140
    legend = np.zeros((legend_h, W, 3), np.uint8)
    f = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(legend, "Legend", (20, 34), f, 1.0, (255,255,255), 2)

    sw = [
        ((150,10),(185,40),(0,0,255),   "Focal"),
        ((200,10),(235,40),(255,0,255), "Subject"),
        ((250,10),(285,40),(0,255,255), "Lines"),
        ((300,10),(335,40),(255,128,0), "Texture"),
        ((350,10),(385,40),(255,0,255), "ΔE"),
        ((400,10),(435,40),(255,255,255),"Thirds"),
        ((450,10),(485,40),(0,255,255), "Golden"),
        ((500,10),(535,40),(0,150,255), "Light→"),
        ((550,10),(585,40),(0,255,255), "Vanishing"),
        ((600,10),(635,40),(200,200,200),"Hue"),
        ((650,10),(685,40),(180,180,180),"Texture dir")
    ]
    for (x1,y1),(x2,y2), col, label in sw:
        if x2 < W - 10:
            cv2.rectangle(legend, (x1,y1), (x2,y2), col, -1)
            cv2.putText(legend, label, (x2+8, y2), f, 0.55, (230,230,230), 1)

    # hue & orientation bars on right
    hue_bar = make_hue_bar(M["color"]["hue_hist"], width=min(480, max(360, W//2)), height=44)
    ori_bar = make_orientation_bar(M["texture"]["gabor_angles"], M["texture"]["gabor_energy"],
                                   width=min(480, max(360, W//2)), height=44)
    hb_h, hb_w = hue_bar.shape[:2]; ob_h, ob_w = ori_bar.shape[:2]
    x0 = max(0, W - max(hb_w, ob_w) - 20)
    hue_bar_resized = hue_bar[:, :min(hb_w, W - x0)]
    ori_bar_resized = ori_bar[:, :min(ob_w, W - x0)]
    y1,y2 = 10, 10 + hb_h; x1,x2 = x0, x0 + hue_bar_resized.shape[1]
    if y2 <= legend.shape[0] and x2 <= legend.shape[1]:
        legend[y1:y2, x1:x2] = hue_bar_resized
    y1b,y2b = 10 + hb_h + 5, 10 + hb_h + 5 + ob_h
    x1b,x2b = x0, x0 + ori_bar_resized.shape[1]
    if y2b <= legend.shape[0] and x2b <= legend.shape[1]:
        legend[y1b:y2b, x1b:x2b] = ori_bar_resized

    final = np.vstack([to_uint8(safe_gamma(comp_rgb)), legend])
    final = final.astype(np.uint8)
    return final

# -----------------------------
# Main pipeline (deep fidelity)
# -----------------------------
def analyze_artwork_high_fidelity(path, verbose=True, thorough=True):
    t0 = time.time()
    if verbose: print(f"🖼️ Loading image: {path}")
    img = load_rgb(path)
    H0, W0 = img.shape[:2]
    if verbose: print(f"   • Image size: {W0}×{H0}")

    # Upscale small inputs to ~1.8MP for fidelity
    if thorough:
        img, scale = resize_min_pixels(img, target_pixels=1_800_000)
        H, W = img.shape[:2]
        if verbose and (H!=H0 or W!=W0):
            print(f"   • Rescaled for fidelity: {W}×{H}")
    else:
        H, W = H0, W0

    # Color (multi-k ensemble)
    if verbose: print("🎨 Deep color clustering (k=6,8,10)…", end="", flush=True)
    color = analyze_color_deep_ensemble(img, ks=(6,8,10))
    if verbose: print(f" ✓ (selected k={color['k_selected']})")

    # Texture (expanded Gabor pyramid)
    if verbose: print("🪶 Texture analysis (Gabor pyramid)…", end="", flush=True)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    texture = gabor_energy_pyramid(gray, scales=(3,5,7,9,11,13), orientations=12)
    if verbose: print(" ✓")

    # Saliency (two-scale)
    if verbose: print("🔥 Saliency mapping (two-scale)…", end="", flush=True)
    sal = analyze_saliency_deep(img)
    if verbose: print(" ✓")

    # Composition (multi-sweep Hough + VP clustering)
    if verbose: print("📐 Composition metrics…", end="", flush=True)
    comp = analyze_composition_deep(img, sal["focal"])
    if verbose: print(" ✓")

    # Illumination
    if verbose: print("💡 Illumination gradients…", end="", flush=True)
    illum = illumination(img)
    if verbose: print(" ✓")

    # Composite
    if verbose: print("✨ Building composite visualization…", end="", flush=True)
    M = dict(color=color, texture=texture, saliency=sal, composition=comp, illumination=illum)
    composite = build_composite(img, M)
    if verbose: print(" ✓")

    # Outputs
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)
    out_png = os.path.join(out_dir, "overlay_final_composite.png")
    cv2.imwrite(out_png, composite)

    # Curatorial text
    if verbose: print("📜 Writing curatorial summary…", end="", flush=True)
    micro, narrative = generate_curatorial_summary(M)
    out_txt = os.path.join(out_dir, "curatorial_summary.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("--- MICRO SUMMARY ---\n")
        f.write(micro + "\n\n--- CURATORIAL SUMMARY ---\n")
        f.write(narrative + "\n")
    if verbose: print(" ✓")

    # Metrics JSON
    if verbose: print("🧾 Saving metrics.json…", end="", flush=True)
    metrics_py = make_json_safe(M)
    out_json = os.path.join(out_dir, "metrics.json")
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(metrics_py, f, indent=2)
    if verbose: print(" ✓")

    if verbose:
        print(f"\n✅ Deep analysis done in {time.time()-t0:.2f}s (thorough mode)")
        print(f"📁 Results saved to: {out_dir}")

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    # ⬇️ Set your image path here
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/2B--glory%20days.jpg"
    analyze_artwork_high_fidelity(IMAGE_PATH, verbose=True, thorough=True)


🖼️ Loading image: /Users/alievanayasso/Documents/SlowMA/2B--glory%20days.jpg
   • Image size: 534×369
   • Rescaled for fidelity: 1613×1115
🎨 Deep color clustering (k=6,8,10)… ✓ (selected k=6)
🪶 Texture analysis (Gabor pyramid)… ✓
🔥 Saliency mapping (two-scale)… ✓
📐 Composition metrics… ✓
💡 Illumination gradients… ✓
✨ Building composite visualization… ✓
📜 Writing curatorial summary… ✓
🧾 Saving metrics.json… ✓

✅ Deep analysis done in 16.79s (thorough mode)
📁 Results saved to: /Users/alievanayasso/Documents/SlowMA/2B--glory%20days_analysis
